In [4]:
import tiktoken

# load encoding directly
#encoding = tiktoken.get_encoding("cl100k_base")
#or load encoding for a specific model
encoding = tiktoken.encoding_for_model("gpt-4o-mini")
print("Encoding name:", encoding.name)
# encode text to tokens
tokens = encoding.encode("Hello, devs 😁! Welcome to the 1st AI crash course.")
print("Encoded tokens:", tokens)

Encoding name: o200k_base
Encoded tokens: [13225, 11, 3947, 82, 22861, 223, 0, 26501, 316, 290, 220, 16, 302, 20837, 20840, 4165, 13]


In [5]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

for encoding_name in ["o200k_base", "cl100k_base"]:
    num_tokens = num_tokens_from_string("Hello, devs 😁! Welcome to the 1st AI crash course.", encoding_name)
    print(f"Number of tokens in encoding '{encoding_name}':", num_tokens)

Number of tokens in encoding 'o200k_base': 17
Number of tokens in encoding 'cl100k_base': 16


In [6]:

# Compare different encodings
# note: convert single tokens back to bytes for token that are out of utf-8 range
def compare_encodings(sentence: str) -> None:
    print(f'\nsentence: "{sentence}"')
    for encoding_name in ["r50k_base", # used by older models like babbage
                          "p50k_base", # used by codex models like text-davinci-003
                          "cl100k_base", # used by models like gpt-4, text-embedding-3-small, text-embedding-3-large
                          "o200k_base" # used by models like gpt-4o-mini
                          ]:
        encoding = tiktoken.get_encoding(encoding_name)
        token_integers = encoding.encode(sentence)
        num_tokens = len(token_integers)
        token_bytes = [encoding.decode_single_token_bytes(token) for token in token_integers]
        print()
        print(f"{encoding_name}: {num_tokens} tokens")
        print(f"token integers: {token_integers}")
        print(f"token bytes: {token_bytes}")
        print(f"decoded back: {encoding.decode(token_integers)}")
        print("-" * 40)
compare_encodings("Hello, devs 😁! Welcome to the 1st AI crash course.")      
print("\n" + "="*80 + "\n")
compare_encodings("お誕生日おめでとうございます！")  


sentence: "Hello, devs 😁! Welcome to the 1st AI crash course."

r50k_base: 15 tokens
token integers: [15496, 11, 41470, 30325, 223, 0, 19134, 284, 262, 352, 301, 9552, 7014, 1781, 13]
token bytes: [b'Hello', b',', b' devs', b' \xf0\x9f\x98', b'\x81', b'!', b' Welcome', b' to', b' the', b' 1', b'st', b' AI', b' crash', b' course', b'.']
decoded back: Hello, devs 😁! Welcome to the 1st AI crash course.
----------------------------------------

p50k_base: 15 tokens
token integers: [15496, 11, 41470, 30325, 223, 0, 19134, 284, 262, 352, 301, 9552, 7014, 1781, 13]
token bytes: [b'Hello', b',', b' devs', b' \xf0\x9f\x98', b'\x81', b'!', b' Welcome', b' to', b' the', b' 1', b'st', b' AI', b' crash', b' course', b'.']
decoded back: Hello, devs 😁! Welcome to the 1st AI crash course.
----------------------------------------

cl100k_base: 16 tokens
token integers: [9906, 11, 81169, 27623, 223, 0, 20776, 311, 279, 220, 16, 267, 15592, 10121, 3388, 13]
token bytes: [b'Hello', b',', b' devs', b' \xf

In [21]:
def num_tokens_from_messages(messages, model="gpt-4o-mini") -> int:
    """Return the number of tokens used by a list of messages."""
    tokens_per_message = 3
    tokens_per_name = 1
    num_tokens = 0
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using o200k_base encoding.")
        encoding = tiktoken.get_encoding("o200k_base")    
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

In [39]:
from dotenv import load_dotenv
load_dotenv("../.env")
from openai import OpenAI
import os

client = OpenAI()

example_messages = [
    {
        "role": "system",
        "content": "You are a helpful, pattern-following assistant that translates corporate jargon into plain English.",
    },
    {
        "role": "user",
        "content": "This late pivot means we don't have time to boil the ocean for the client deliverable.",
    },
]

for model in [    
    "gpt-4",
    "gpt-4o-mini"
     ]:
    print(model)
    # example token count from the function defined above
    print(f"{num_tokens_from_messages(example_messages, model)} prompt tokens counted by tiktoken.")
    # example token count from the OpenAI API
    response = client.chat.completions.create(model=model,
    messages=example_messages,
    temperature=0,
    max_tokens=10)
    print(f'{response.usage.prompt_tokens} prompt tokens counted by the OpenAI API.')
    print(f'{response.usage.completion_tokens} completion tokens counted by the OpenAI API.')
    print(f'{response.usage.total_tokens} total tokens counted by the OpenAI API.')
    print()

gpt-4
48 prompt tokens counted by tiktoken.
48 prompt tokens counted by the OpenAI API.
10 completion tokens counted by the OpenAI API.
58 total tokens counted by the OpenAI API.

gpt-4o-mini
46 prompt tokens counted by tiktoken.
46 prompt tokens counted by the OpenAI API.
10 completion tokens counted by the OpenAI API.
56 total tokens counted by the OpenAI API.



In [67]:
from openai import OpenAI
import base64

client = OpenAI() 

# image generation with text-only input and image output using tool call
response = client.responses.create(
    model="gpt-4.1-mini",
    input="Generate a labeled diagram illustrating the structure of a flower. The diagram shows the internal and external parts of a flower with labels pointing to each part.",
    tools=[{"type": "image_generation"}]
)

# save image
image_data = [output.result for output in response.output if output.type == "image_generation_call"]
if image_data:
    image_base64 = image_data[0]
    with open("./tmp/flower_diagram.png", "wb") as f:
        f.write(base64.b64decode(image_base64))

display(response.usage.__dict__)        

{'input_tokens': 2063,
 'input_tokens_details': InputTokensDetails(cached_tokens=0),
 'output_tokens': 147,
 'output_tokens_details': OutputTokensDetails(reasoning_tokens=0),
 'total_tokens': 2210}

```
{'input_tokens': 2063,
 'input_tokens_details': InputTokensDetails(cached_tokens=0),
 'output_tokens': 147,
 'output_tokens_details': OutputTokensDetails(reasoning_tokens=0),
 'total_tokens': 2210}
 ```

- Token usage only measures text processed by the model, not the binary data of the image.
1. Text in → tokens counted.
    prompt text, tool specification, and system context make up the 2,063 input tokens.

2. Text out → tokens counted.
    model’s response (which includes the tool call request and metadata, not the base64 image itself) adds 147 output tokens.

3. Image bytes → not tokens.
    - image generation done by the image_generation tool consumes compute resources (typically billed separately), but its binary output is not part of the token accounting
    - tokens apply only to the LLM’s text reasoning and tool-calling layer.

- multi-modal flow (may change depending on provider implementation, models, tools, etc.):
```
prompt → LLM (text tokens)
   ↳ tool call instruction → image generator (no tokens)
      ↳ image returned as base64 → 1_666_983 bytes delivered: ~1.59 MB
```

- Key takeaways:

    . `Monitor` tokens consumption but also be aware of the media generation costs
    
    . Tokens count != price